## 환경설정

아래 설치 방법을 참고하여 `uv` 를 설치합니다.

**uv 설치 방법**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

**의존성 설치**

```bash
uv pip install -r requirements.txt
```

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="/mnt/hdd/hyeontae/langgraph-mcp-agents/.env",
            override=True)

True

## MultiServerMCPClient

`async with`로 일시적인 Session 연결을 생성 후 해제

In [ ]:
import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from utils import ainvoke_graph, astream_graph
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_output_tokens=1024,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)

client =  MultiServerMCPClient(
    {
        "weather": {
            # 서버의 포트와 일치해야 한다.
            "url": "http://localhost:8005/sse", # local에서 서버 올려서 8005번과 통신하겠다.
            "transport": "sse", # sse 방식으로 통신
        }
    }
)

print(await client.get_tools())
agent = create_react_agent(model, tools=await client.get_tools())
answer = await astream_graph(agent, {"messages" : "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f455f63a8e0>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - 

다음의 경우에는 session이 닫혔기 때문에 도구에 접근할 수 없는 것을 확인할 수 있습니다.

## Stdio 통신 방식

Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.

- 통신을 위해 표준 입력/출력 사용

참고: 아래의 python 경로는 수정하세요!

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_output_tokens=1024,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)

# StdIO 서버 파라미터 설정
# - command : Python 인터프리터 경로
# - args : 실행할 MCP 서버 스크립트
server_params = StdioServerParameters(
    command = "/mnt/hdd/Anaconda3/mcp/bin/python",
    args = ["mcp_server_local.py"]
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()
        
        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)
        
        # 에이전트 생성
        agent = create_react_agent(model, tools=tools)
        
        # 에이전트 응답 스트리밍
        await astream_graph(agent,
                           {"messages": "서을의 날씨는 어떠니?"})

## RAG 를 구축한 MCP 서버 사용

- 파일: `mcp_server_rag.py`

사전에 langchain 으로 구축한 `mcp_server_rag.py` 파일을 사용합니다.

stdio 통신 방식으로 도구에 대한 정보를 가져옵니다. 여기서 도구는 `retriever` 도구를 가져오게 되며, 이 도구는 `mcp_server_rag.py` 에서 정의된 도구입니다. 이 파일은 사전에 서버에서 실행되지 **않아도** 됩니다.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from utils import astream_graph
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_output_tokens=1024,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)

# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="/mnt/hdd/Anaconda3/mcp/bin/python",
    args=["mcp_server_rag.py"]
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()
        
        # MCP 도구 로드 ( 여기서는 Retriever 도구 )
        tools = await load_mcp_tools(session)

        # 에이전트 생성 및 실행
        agent = create_react_agent(model, tools=tools)
        
        # 에이전트 응답 스트리밍
        await astream_graph(
            agent,
            {"messages": "삼성전자가 개발한 생성형 AI의 이름을 검색해줘"}
        )

## SSE 방식과 StdIO 방식 혼합 사용

- 파일: `mcp_server_rag.py` 는 StdIO 방식으로 통신
- `langchain-dev-docs` 는 SSE 방식으로 통신

SSE 방식과 StdIO 방식을 혼합하여 사용합니다.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_output_tokens=1024,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)

# 1. 다중 서버 MCP 클라이언트 생성
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "/mnt/hdd/Anaconda3/mcp/bin/python",
            "args": ["mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
        
        "langchain-dev-docs": {
            "url": "https://teddynote.io/mcp/langchain/sse",
            # SSE(Server-Sent Events) 방식으로 통신
            "transport": "sse",
        },
    }
)

# 2. 비동기 컨텍스트 매니저를 통한 명시적 연결 초기화
await client.get_tools()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = """You are a smart agent.
Use 'retrieve' tool to search AI related documents and answer questions.
Use 'langchain-dev-docs' tools to search on langchain / langgraph related documents and answer questino.
Answer in Korean."""

agent = create_react_agent(
    model, 
    await client.get_tools(), 
    prompt=prompt, 
    checkpointer=MemorySaver()
)

구축해 놓은 `mcp_server_rag.py` 에서 정의한 `retriever` 도구를 사용하여 검색을 수행합니다.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "retrieve 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘."
    },
    config=config
)

이번에는 `langchain-dev-docs` 도구를 사용하여 검색을 수행합니다.

In [ ]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {"messages": "langgraph-dev-docs 도구를 사용해서 self-rag 코드를 작성해 주세요(langgraph로)."},
    config=config,
)

`MemorySaver` 를 사용하여 단기 기억을 유지합니다. 따라서, multi-turn 대화도 가능합니다.

In [ ]:
await astream_graph(
    agent, {"messages": "이전의 내용을 bullet point 로 요약해줘"}, config=config
)

## LangChain 에 통합된 도구 + MCP 도구

여기서는 LangChain 에 통합된 도구를 기존의 MCP 로만 이루어진 도구와 함께 사용이 가능한지 테스트 합니다.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily 검색 도구를 초기화합니다. (news 타입, 최근 3일 내 뉴스)
tavily = TavilySearchResults(max_results=3, topic="news", days=3)

# 기존의 MCP 도구와 함께 사용합니다.
tools = await client.get_tools() + [tavily]

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=30, thread_id=2)

prompt = "You are a smart agent with various tools. Answer questions in Korean."

agent = create_react_agent(
    model,
    tools,
    prompt=prompt,
    checkpointer=MemorySaver()
)

새롭게 추가한 `tavily` 도구를 사용하여 검색을 수행합니다.

In [ ]:
await astream_graph(
    agent, 
    {"messages": "아이폰 17 관련한 오늘 뉴스 찾아줘"},
    config=config
)

In [ ]:
await astream_graph(
    agent,
    {
        "messages": "`retrieve` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)

## Smithery 에서 제공하는 MCP 서버

- 링크: https://smithery.ai/

사용한 도구 목록은 아래와 같습니다.

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - 구조화된 사고 프로세스를 통해 역동적이고 성찰적인 문제 해결을 위한 도구를 제공하는 MCP 서버
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - 다양한 편집 기능으로 터미널 명령을 실행하고 파일을 관리하세요. 코딩, 셸 및 터미널, 작업 자동화

**참고**

- smithery 에서 제공하는 도구를 JSON 형식으로 가져올때, 아래의 예시처럼 `"transport": "stdio"` 로 꼭 설정해야 합니다.

In [1]:
def load_mcp_config():
    """현재 디렉토리의 MCP 설정 파일을 로드합니다."""
    try:
        with open("./config.json", "r") as f:
            return json.load(f)
    except Exception as e:
        print(f"설정 파일을 읽는 중 오류 발생: {str(e)}")
        return None

def create_server_config():
    """MCP 서버 설정을 생성합니다."""
    config = load_mcp_config()
    server_config = {}

    if config and "mcpServers" in config:
        for server_name, server_config_data in config["mcpServers"].items():
            # command가 있으면 stdio 방식
            if "command" in server_config_data:
                server_config[server_name] = {
                    "command": server_config_data.get("command"),
                    "args": server_config_data.get("args", []),
                    "transport": "stdio",
                }
            # url이 있으면 sse 방식
            elif "url" in server_config_data:
                server_config[server_name] = {
                    "url": server_config_data.get("url"),
                    "transport": "sse",
                }

    return server_config


In [2]:
import json
import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="/mnt/hdd/hyeontae/langgraph-mcp-agents/.env",
            override=True)

# LLM 모델 초기화
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_output_tokens=1024,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)

server_config = create_server_config()
client = MultiServerMCPClient(server_config)

# agent = create_react_agent(model, await client.get_tools())

# query ="현재 경로를 포함한 하위 폴더 구조를 tree 로 그려줘. 단, .venv 폴더는 제외하고 출력해줘."

# response = await agent.invoke({"messages":query})
# print(response)

In [ ]:
tools = await client.get_tools()

In [ ]:
import os
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="/mnt/hdd/hyeontae/langgraph-mcp-agents/.env",
            override=True)

# LLM 모델 초기화
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_output_tokens=1024,
    google_api_key=os.environ.get("GOOGLE_API_KEY"),
)
# 1. 클라이언트 생성
client_x = MultiServerMCPClient(
    {
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "c8974619-e333-499e-ae13-3c6e7e5c0c31"
            ],
            "transport" : "stdio"
        },
    
        "document-retriever": {
            "command": "/mnt/hdd/Anaconda3/mcp/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
    }
)

# await client.__aenter__()

In [ ]:
await client_x.get_tools()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=30, thread_id=3)
agent = create_react_agent(model, await client.get_tools(), checkpointer=MemorySaver())